# Unigram tokenizer example

## Loading Vietnamese dataset

In [14]:
import datasets

vi_dataset = datasets.load_dataset("facebook/belebele", "vie_Latn", split="test")
vi_dataset

Dataset({
    features: ['link', 'question_number', 'flores_passage', 'question', 'mc_answer1', 'mc_answer2', 'mc_answer3', 'mc_answer4', 'correct_answer_num', 'dialect', 'ds'],
    num_rows: 900
})

In [15]:
vi_dataset[0]

{'link': 'https://en.wikibooks.org/wiki/Accordion/Right_hand',
 'question_number': 1,
 'flores_passage': 'Hãy đảm bảo tay của bạn được thư giãn nhất có thể khi nhấn tất cả các nốt nhạc chính xác - và cố gắng đừng làm nhiều cử động ngón tay không cần thiết. Chơi đàn theo cách này sẽ giúp bạn giảm mất sức tối đa. Hãy nhớ rằng, bạn không cần phải dùng nhiều lực để ấn phím nhằm tăng âm lượng như khi chơi đàn piano. Trên đàn phong cầm, để tăng âm lượng, bạn sử dụng ống thổi tác động mạnh hơn hoặc nhanh hơn.',
 'question': 'Theo đoạn văn, đâu không được coi là lời khuyên đúng để chơi được đàn phong cầm?',
 'mc_answer1': 'Để tăng âm lượng, dùng thêm nhiều lực khi ấn phím',
 'mc_answer2': 'Hạn chế tối thiểu động tác thừa để giữ sức',
 'mc_answer3': 'Chú ý khi nhấn các nốt nhạc phải giữ cho tay được thư giãn nhất có thể',
 'mc_answer4': 'Tăng tốc độ di chuyển hộp xếp để tăng âm lượng',
 'correct_answer_num': '1',
 'dialect': 'vie_Latn',
 'ds': '2023-06-01'}

## Training Unigram tokenizer

In [16]:
from tokenizers import Tokenizer, decoders, models, normalizers, pre_tokenizers, trainers
tokenizer = Tokenizer(models.Unigram())

normalizer = normalizers.Sequence([normalizers.NFD(), 
                                normalizers.Lowercase(),
                                normalizers.Strip()])
tokenizer.normalizer = normalizer

pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(), 
                                        pre_tokenizers.Digits(individual_digits=True)])
tokenizer.pre_tokenizer = pre_tokenizer

# tokenizer.decoder = decoders.ByteLevel()
trainer = trainers.UnigramTrainer(special_tokens=["<BLANK>", "<S>", "<UNK>"],
                                    show_progress=True, 
                                    continuing_subword_prefix="_")

Ignored unknown kwargs option continuing_subword_prefix


In [17]:
def batch_iterator(batch_size=1000):
    for i in range(0, len(vi_dataset), batch_size):
        yield vi_dataset[i : i + batch_size]["flores_passage"]
        
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(vi_dataset))

In [18]:
tokenizer.get_vocab_size()

2090

In [19]:
tokenizer.get_vocab()

{'chồ': 1018,
 'uy': 937,
 'và': 19,
 'sla': 1577,
 'uri': 1935,
 'giá': 136,
 'christ': 1609,
 'hú': 609,
 'hắn': 1168,
 'sá': 164,
 '%': 738,
 'vati': 1005,
 'asu': 2062,
 'pati': 1213,
 'ge': 1957,
 'só': 444,
 'hò': 643,
 'anf': 1859,
 'bề': 805,
 'à': 88,
 'aria': 1017,
 'đậ': 979,
 'mậ': 952,
 'ấ': 109,
 '1': 45,
 '”': 334,
 'trô': 757,
 'waz': 1306,
 'ju': 1154,
 'lộ': 611,
 'es': 673,
 'af': 793,
 'bur': 1770,
 'vu': 631,
 'ngạ': 736,
 'iế': 703,
 'lán': 1281,
 'ks': 1974,
 'yê': 354,
 'pale': 1203,
 'ră': 437,
 'dặ': 450,
 'ook': 1588,
 'hạc': 839,
 'll': 1790,
 'm': 13,
 'qua': 90,
 'đí': 760,
 'tai': 1197,
 's': 30,
 'cụ': 612,
 'for': 946,
 'rắ': 1415,
 'kalaall': 1294,
 'khi': 52,
 'giu': 472,
 'như': 47,
 'tron': 60,
 'eren': 1836,
 'rư': 1911,
 'xoa': 686,
 'lưỡ': 807,
 'chọ': 632,
 'anne': 2023,
 'john': 656,
 'foo': 1162,
 'vann': 1779,
 'phụ': 288,
 '6': 163,
 'enst': 1766,
 'tric': 1866,
 'bó': 1245,
 'tụ': 668,
 'ling': 17

In [20]:
tokenizer.save('vi-wiki-tokenizer/unigram-tokenizer.json')